In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd /home/vagrawal/RVT-CO2/code

In [ ]:
import os
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader
from torchsummary import summary
from models import *
from datasets import resp_co2_dataset
import seaborn as sns

from train_utils import Trainer
import utils

In [ ]:
%matplotlib notebook
plt.rcParams["figure.figsize"] = [9,4]

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = [16,4]

In [ ]:
def stats_bean_plot(corr_stats):
    corr_stats_copy = corr_stats.copy()    
    del corr_stats_copy['mape_co2'], corr_stats_copy['z_co2'], corr_stats_copy['z_petco2']
    df_stats = pd.DataFrame.from_dict(corr_stats_copy)
    sns.violinplot(data = pd.DataFrame.from_dict(corr_stats_copy))

In [ ]:
dict_stats = {"split_and_model_name": [],
                "mean_corr_co2" : [],
                "mean_corr_petco2" : [],
                "mean_z_co2" : [],
                "mean_z_petco2" : [],
                "mean_mse_co2" : [],
                "mean_mse_petco2" : [],
                "mean_mae_co2" : [],
                "mean_mae_petco2" : [],
                "mean_mape_co2" : [],
                "mean_mape_petco2" : [],
                "std_corr_co2" : [],
                "std_corr_petco2" : [],
                "std_z_co2" : [],
                "std_z_petco2" : [],
                "std_mse_co2" : [],
                "std_mse_petco2" : [],
                "std_mae_co2" : [],
                "std_mae_petco2" : [],
                "std_mape_co2" : [],
                "std_mape_petco2" : [],}

def add_mean_std_stats(current_model_name, corr_stats):
    current_split_and_model_name = str(data_split_number) + '-' + current_model_name
    if current_split_and_model_name not in dict_stats[f"split_and_model_name"]:
        dict_stats["split_and_model_name"].append(current_split_and_model_name) 
        for key, value in corr_stats.items():
            dict_stats[f'mean_{key}'].append(round(np.mean(value), 3)) 
            dict_stats[f'std_{key}'].append(round(np.std(value), 3))

In [ ]:
dict_models = {
    'linear_without_tanh': linear_without_tanh(in_channels=1, out_channels=1),
    'linear_with_tanh': linear_with_tanh(in_channels=1, out_channels=1),
    'conv_1_layer': conv_1_layer(in_channels=1, out_channels=1),
    'conv_2_layer': conv_2_layer(in_channels=1, out_channels=1),
    'conv_4_layer': conv_4_layer(in_channels=1, out_channels=1),
    'conv_6_layer': conv_6_layer(in_channels=1, out_channels=1),
    'conv_1_layer_weightedLoss': conv_1_layer(in_channels=1, out_channels=1),
    'conv_2_layer_weightedLoss': conv_2_layer(in_channels=1, out_channels=1),
    'conv_4_layer_weightedLoss': conv_4_layer(in_channels=1, out_channels=1),
    'conv_6_layer_weightedLoss': conv_6_layer(in_channels=1, out_channels=1),
}

In [ ]:
baycrest_root = '/home/vagrawal/RVT-CO2/data/raw_physio_backup-biopac-20180417/preprocessed'
apply_co2_norm = True

In [ ]:
# Config
train_models = False
save_models = False
save_logs = False
load_models = True
phases_test = ['test']

date = '20201011'
save_root = os.path.join('./saved_models', date)

In [ ]:
for data_split_number in [0,1,2,3,4]:
    txt_root = os.path.join(baycrest_root, 'splits_train_test', str(data_split_number))
    dataloader_train = resp_co2_dataset(txt_root, resp_filename = 'resp_train.txt', co2_filename = 'co2_train.txt',apply_co2_norm=apply_co2_norm)
    dataloader_test = resp_co2_dataset(txt_root, resp_filename = 'resp_test.txt', co2_filename = 'co2_test.txt',apply_co2_norm=apply_co2_norm)

    dataloaders = {
        'train':DataLoader(dataloader_train, batch_size=1, shuffle=False),
        'test':DataLoader(dataloader_test, batch_size=1, shuffle=False),
    }
    
    split_number = data_split_number
    for current_model_name in dict_models.keys():

        current_model = dict_models[current_model_name]
        save_dir = os.path.join(save_root, current_model_name)
        save_path = os.path.join(save_dir, f'{split_number}.pth.gz')
        
        optimizer_lr = 0.01
        net = Trainer(current_model, dataloaders, optimizer_lr)
        net.load_model(save_path)  
        corr_stats = net.nnTest(phases = phases_test, plots=False, output_smoothing=True, output_stdnorm = False)
        add_mean_std_stats(current_model_name, corr_stats)

In [ ]:
df_stats = pd.DataFrame(dict_stats)

In [ ]:
df_stats

In [ ]:
df_stats.to_csv(f'saved_logs/manuscript/{date}_train.csv')